**Пример загрузки из Postgres в Hive при помощи onETL**

Имопрт mtspark'a для создания сессии и onETL.

In [1]:
from mtspark import init_spark

from onetl.connection.db_connection import Postgres, Hive
from onetl.reader.db_reader import DBReader
from onetl.writer.db_writer import DBWriter

Создаем сессию с помощью **mtspark**.

**!! Обратите внимание, что нужно прокинуть Jar файл для коннекта с базой !!**

In [2]:
spark = init_spark({"appName": "spark-app-name", "spark.jars":"/path/to/*.jar"})

:: loading settings :: file = /opt/anaconda/envs/jh-py37-conda/lib/python3.7/site-packages/mtspark/config/ivysettings.xml


Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).


Создаем классы **Connection** к **Postgres** и **Hive**.

Также создаем словарь с опциями для jdbc.

In [3]:
jdbc_options = {'sessionInitStatement': 'select 300', 'fetchsize': '100'}
postgres = Postgres(host='test-db-vip.msk.mts.ru', login='user', password='pass', database='your_db', spark=spark)
hive = Hive(spark=spark)

Класс **DBReader** для чтения по **jdbc**.

Метод **run** возвращает **dataFrame**

In [4]:
reader = DBReader(connection=postgres, table='schema.table_name', jdbc_options=jdbc_options)
table_df = reader.run()

Класс **DBWriter** для записи по **jdbc** или в **Hive**.

Методу **run** передается **dataFrame** для записи.

In [5]:
writer = DBWriter(connection=hive, table='pets_sb.test_onetl', format='parquet', mode='overwrite')
writer.run(table_df)